In [ ]:
import sys
import os
import shutil
import time
from pathlib import Path
import csv
from math import *
import numpy as np
import matplotlib.pyplot as plt
import cv2

from utils import get_abs_path

In [ ]:
def cv2_imshow(img):
    img = np.concatenate((img[:,:,2:3], img[:,:,1:2], img[:,:,0:1]), axis=2)
    plt.imshow(img)
    plt.show()


def rand_point_on_map(map_size, offset):

    min_pos = offset
    max_x_pos = map_size[0]-offset
    max_y_pos = map_size[1]-offset

    x_pos = np.random.randint(low = min_pos, high = max_x_pos, size = 1)
    y_pos = np.random.randint(low = min_pos, high = max_y_pos, size = 1)
    return int(x_pos), int(y_pos)


def create_start_point(img, map_size, offset=3):

    x_pos, y_pos = rand_point_on_map(map_size, offset)

    occupied_pos = 255
    while img[y_pos,x_pos,0] == occupied_pos:
        x_pos, y_pos = rand_point_on_map(map_size, offset)

    #cv2.rectangle(img, (x_pos-2, y_pos-2), (x_pos+2, y_pos+2), (0,255,0), -1)
    return x_pos, y_pos


def create_end_point(img, start_x_pos, start_y_pos, map_size, min_distance, offset=3):

    x_pos, y_pos = rand_point_on_map(map_size, offset)
    occupied_pos = 255
    while True:
        x_pos, y_pos = rand_point_on_map(map_size, offset)

        if img[y_pos,x_pos,0] != occupied_pos:
            if sqrt( pow(start_x_pos-x_pos,2) + pow(start_y_pos-y_pos,2) ) > float(min_distance):
                break

    #cv2.rectangle(img, (x_pos-2, y_pos-2), (x_pos+2, y_pos+2), (0,255,0), -1)
    return x_pos, y_pos


def create_rect(img, x_pos, y_pos, width, height):
    x_pos, y_pos, width, height = int(x_pos), int(y_pos), int(width), int(height)
    cv2.rectangle(img, (x_pos, y_pos), (x_pos+width, y_pos+height), (255,0,0), -1)


def add_obstacles(img, rect_count, rect_min, rect_max):
    for i in range(rect_count):
        rect_width = np.random.randint(low = rect_min, high = rect_max, size = 1)
        rect_heigth = np.random.randint(low = rect_min, high = rect_max, size = 1)
        pos = np.random.randint(low = 0, high = map_width-rect_width, size = 4)
        create_rect(img, pos[0], pos[1], rect_width, rect_heigth)

In [ ]:
# image size
map_width = 120
map_height = 120
map_size = [map_width, map_height]

# rect settings
rect_count = 7
rect_min = 15
rect_max = 25

In [ ]:
# generate test image
img = np.zeros((map_width, map_height, 3), np.uint8)
add_obstacles(img, rect_count, rect_min, rect_max)

start_x_pos, start_y_pos = create_start_point(img, map_size)
end_x_pos, end_y_pos = create_end_point(img, start_x_pos, start_y_pos, map_size, min_distance=90)
cv2_imshow(img)

In [ ]:
# (first_map_num, last_map_num, num_paths)
data_settings = {
    'encoder': (0, 10000, 1),
    'train': (0, 50, 200),
    'valid_unknown': (0, 50, 40),
    'test_unknown':  (0, 50, 40),
    'valid_known': (200),
    'test_known':  (200)
}
project_path = get_abs_path(1)
results_path = project_path + '/data/'
maps_points_filename = 'maps_points.csv'

# remove old point files
# os.remove(results_path+'train/'+maps_points_filename)
# os.remove(results_path+'valid/'+maps_points_filename)
# os.remove(results_path+'test/'+maps_points_filename)

# create dirs
Path(results_path+'encoder_images/').mkdir(parents=True, exist_ok=True)
Path(results_path+'train/maps/').mkdir(parents=True, exist_ok=True)
Path(results_path+'valid/known_maps/').mkdir(parents=True, exist_ok=True)
Path(results_path+'valid/unknown_maps/').mkdir(parents=True, exist_ok=True)
Path(results_path+'test/known_maps/').mkdir(parents=True, exist_ok=True)
Path(results_path+'test/unknown_maps/').mkdir(parents=True, exist_ok=True)

In [ ]:
def generate_maps(results_path, first_map_num, last_map_num, num_paths):
    for n in range(first_map_num, last_map_num):
        # generate image
        image = np.zeros((map_width, map_height, 3), np.uint8)

        # add obstacles
        add_obstacles(image, rect_count, rect_min, rect_max)

        # save image
        number = str(n)
        while len(number) < 6:
            number = '0' + number
        map_name = 'map_' + number + '.png'

        if not cv2.imwrite(results_path + map_name, image):
            raise Exception('Could not write image')

        for i in range(num_paths):
            # create start and end points
            img = np.copy(image)
            start_x_pos, start_y_pos = create_start_point(img, map_size)
            end_x_pos, end_y_pos = create_end_point(img, start_x_pos, start_y_pos, map_size, min_distance=65)

            # create map data
            number = str(i)
            while len(number) < 6:
                number = '0' + number
            map_data = [map_name[:-4] + '_' + number, str(start_x_pos), str(start_y_pos), str(end_x_pos), str(end_y_pos)]

            # save points to csv
            # print(map_data)
            with open(results_path + maps_points_filename, 'a', newline='') as f:
                writer = csv.writer(f)
                writer.writerows([map_data])


generate_maps(results_path+'encoder_images/',  *data_settings['encoder'])
generate_maps(results_path+'train/maps/', *data_settings['train'])
generate_maps(results_path+'valid/unknown_maps/', *data_settings['valid_unknown'])
generate_maps(results_path+'test/unknown_maps/',  *data_settings['test_unknown'])

In [ ]:
def copy_maps(origin_maps_paths, results_path, num_paths):
    files = os.listdir(origin_maps_paths)
    files = [f for f in files if f.endswith('.png')]
    files = sorted(files)

    for fname in files:

        image_path = os.path.join(origin_maps_paths,fname)
        shutil.copy2(image_path, results_path)

        img = cv2.imread(image_path)

        for i in range(num_paths):

            start_x_pos, start_y_pos = create_start_point(img, map_size)
            end_x_pos, end_y_pos = create_end_point(img, start_x_pos, start_y_pos, map_size, min_distance=65)

            # create map data
            number = str(i)
            while len(number) < 6:
                number = '0' + number
            map_data = [fname[:-4] + '_' + number, str(start_x_pos), str(start_y_pos), str(end_x_pos), str(end_y_pos)]

            # save points to csv
            # print(map_data)
            with open(results_path + maps_points_filename, 'a', newline='') as f:
                writer = csv.writer(f)
                writer.writerows([map_data])



copy_maps(results_path+'train/maps/', results_path+'valid/known_maps/', data_settings['valid_known'])
copy_maps(results_path+'train/maps/', results_path+'test/known_maps/', data_settings['test_known'])